In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
output_directory = '/Users/francescmartiescofet/Documents/ETH/IVT/eqasim-java/switzerland/SA_scenario_output'
last_iter = 5

In [3]:
global_avg = []
global_moving_avg = []
global_successive_avg = []
zonal_avg = []
drt_legs = []
for i in range(last_iter + 1):
    iter_path = os.path.join(output_directory,'testDrtZones','ITERS','it.' + str(i)) + \
                '/' + str(i) + '.'
    global_avg.append(pd.read_csv(iter_path + 'DrtWaitTimeGlobalAvg.csv', sep=';'))
    global_moving_avg.append(pd.read_csv(iter_path + 'DrtWaitTimeGlobalMovingAvg.csv', sep=';'))
    global_successive_avg.append(pd.read_csv(iter_path + 'DrtWaitTimeGlobalSuccessiveAvg.csv', sep=';'))
    zonal_avg.append(pd.read_csv(iter_path + 'DrtWaitTimes.csv', sep=';'))
    drt_legs.append(pd.read_csv(iter_path + 'drt_legs_drt.csv', sep=';'))

link2zones_csv = pd.read_csv(os.path.join(output_directory,'testDrtZones','drt_WayneCountyLink2Zones.csv'), sep=';')
link2zones_csv = link2zones_csv.set_index('link_id')

In [4]:
link2zones = {}
for link_id, zone in link2zones_csv.iterrows():
    link2zones[link_id] = zone.zone.item()

In [5]:
def check_global_avg(global_avg, drt_legs):
    for it, (it_global_avg, it_drt_legs) in enumerate(zip(global_avg, drt_legs)):
        print('Iteration ' + str(it))
        print('Global avg: ' + str(it_global_avg['avg'][0]))
        print('Average from legs: ' + str(it_drt_legs.waitTime.mean()))
        print('\n')

In [6]:
check_global_avg(global_avg, drt_legs)

Iteration 0
Global avg: nan
Average from legs: nan


Iteration 1
Global avg: 70.625
Average from legs: 70.625


Iteration 2
Global avg: 70.56410256410257
Average from legs: 70.56410256410257


Iteration 3
Global avg: 71.62745098039215
Average from legs: 71.62745098039215


Iteration 4
Global avg: 71.70666666666666
Average from legs: 71.70666666666666


Iteration 5
Global avg: 73.35483870967742
Average from legs: 73.35483870967742




In [7]:
def check_global_moving_window_avg(global_moving_avg, drt_legs, moving_window):
    for it, it_global_moving_avg in enumerate(global_moving_avg):
        start = 0
        if it >= moving_window:
            start = it - moving_window + 1
        it_drt_legs = pd.concat([drt_legs[j] for j in range(start, it+1)])
        print('Iteration ' + str(it))
        print('Global moving avg: ' + str(it_global_moving_avg['avg'][0]))
        print('Average from legs: ' + str(it_drt_legs.waitTime.mean()))
        print('\n')
        

In [8]:
check_global_moving_window_avg(global_moving_avg, drt_legs, 2)

Iteration 0
Global moving avg: nan
Average from legs: nan


Iteration 1
Global moving avg: 70.625
Average from legs: 70.625


Iteration 2
Global moving avg: 70.58730158730158
Average from legs: 70.58730158730158


Iteration 3
Global moving avg: 71.16666666666667
Average from legs: 71.16666666666667


Iteration 4
Global moving avg: 71.67460317460318
Average from legs: 71.67460317460318


Iteration 5
Global moving avg: 72.61904761904762
Average from legs: 72.61904761904762




In [9]:
def check_global_successive_avg(global_successive_avg, drt_legs, weight):
    iters_avg = []
    for it_drt_leg in drt_legs:
        iters_avg.append(it_drt_leg.waitTime.mean())
    print(iters_avg)
    iters_successive_avg = []
    for it in range(len(iters_avg)):
        if it == 0:
            iters_successive_avg.append(iters_avg[it])
            continue
        previous_avg = iters_successive_avg[it-1]
        if np.isnan(previous_avg):
            iters_successive_avg.append(iters_avg[it])
            continue
        iters_successive_avg.append((1-weight) * previous_avg + weight * iters_avg[it])
            
    for it, it_global_successive_avg in enumerate(global_successive_avg):
        print('Iteration ' + str(it))
        print('Global successive avg: ' + str(it_global_successive_avg['avg'][0]))
        print('Average from legs: ' + str(iters_successive_avg[it]))
        print('\n')

In [10]:
check_global_successive_avg(global_successive_avg, drt_legs, 0.5)

[nan, 70.625, 70.56410256410257, 71.62745098039215, 71.70666666666666, 73.35483870967742]
Iteration 0
Global successive avg: nan
Average from legs: nan


Iteration 1
Global successive avg: 70.625
Average from legs: 70.625


Iteration 2
Global successive avg: 70.59455128205128
Average from legs: 70.59455128205128


Iteration 3
Global successive avg: 71.11100113122171
Average from legs: 71.11100113122171


Iteration 4
Global successive avg: 71.40883389894418
Average from legs: 71.40883389894418


Iteration 5
Global successive avg: 72.38183630431081
Average from legs: 72.38183630431081




In [11]:
def check_zonal_avg(zonal_avg, drt_legs, link2zones, iter_to_check, time_bins = 100):
    
    it_zonal_avg = zonal_avg[iter_to_check].copy()
    it_drt_legs = drt_legs[iter_to_check].copy()
    print(it_zonal_avg.shape)
    print(it_drt_legs.shape)
    hour_bins = [i*3600 for i in range(0,time_bins)]
    it_drt_legs['time_bin'] = pd.cut(it_drt_legs.departureTime, hour_bins)
    it_drt_legs['zone'] = it_drt_legs['fromLinkId'].map(link2zones)
    it_drt_legs.groupby(['zone','time_bin']).waitTime.mean()
    # Checked some random zones and the avg times match
    print(it_drt_legs.groupby(['zone','time_bin']).waitTime.mean()[113934].iloc[:20])
    print(it_zonal_avg[it_zonal_avg.zone == 113934].iloc[:,:20])

In [12]:
check_zonal_avg(zonal_avg, drt_legs, link2zones, 1)

(31, 101)
(24, 17)
time_bin
(0, 3600]           NaN
(3600, 7200]        NaN
(7200, 10800]       NaN
(10800, 14400]      NaN
(14400, 18000]      NaN
(18000, 21600]      NaN
(21600, 25200]      NaN
(25200, 28800]      NaN
(28800, 32400]    109.0
(32400, 36000]      NaN
(36000, 39600]      NaN
(39600, 43200]      NaN
(43200, 46800]     61.0
(46800, 50400]      NaN
(50400, 54000]      NaN
(54000, 57600]      NaN
(57600, 61200]      NaN
(61200, 64800]      NaN
(64800, 68400]      NaN
(68400, 72000]      NaN
Name: waitTime, dtype: float64
     zone    0    1    2    3    4    5    6    7      8    9   10   11    12  \
5  113934  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  109.0  0.0  0.0  0.0  61.0   

    13   14   15   16   17   18  
5  0.0  0.0  0.0  0.0  0.0  0.0  
